In [1]:
import gym
import ptan
import random

import torch
import torch.optim as optim

from ignite.engine import Engine

from lib import dqn_model, common

NAME = "01_baseline"

In [2]:
random.seed(common.SEED)
torch.manual_seed(common.SEED)
params = common.HYPERPARAMS['pong']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = gym.make("PongNoFrameskip-v4")
env = ptan.common.wrappers.wrap_dqn(env)
env.seed(common.SEED)

[123, 151010689]

In [4]:
net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)

tgt_net = ptan.agent.TargetNet(net)
selector = ptan.actions.EpsilonGreedyActionSelector(
    epsilon=params.epsilon_start)
epsilon_tracker = common.EpsilonTracker(selector, params)
agent = ptan.agent.DQNAgent(net, selector, device=device)

exp_source = ptan.experience.ExperienceSourceFirstLast(
    env, agent, gamma=params.gamma)
buffer = ptan.experience.ExperienceReplayBuffer(
    exp_source, buffer_size=params.replay_size)
optimizer = optim.Adam(net.parameters(),
                       lr = params.learning_rate)

In [6]:
def process_batch(engine, batch):
    optimizer.zero_grad()
    loss_v = common.calc_loss_dqn(
        batch, net, tgt_net.target_model, gamma=params.gamma, device=device)
    loss_v.backward()
    optimizer.step()
    epsilon_tracker.frame(engine.state.iteration)
    if engine.state.iteration % params.target_net_sync == 0:
        tgt_net.sync()
    return {
        "loss": loss_v.item(),
        "epsilon": selector.epsilon,
    }

engine = Engine(process_batch)
common.setup_ignite(engine, params, exp_source, NAME)
engine.run(common.batch_generator(buffer, params.replay_initial,
                                  params.batch_size))

Episode 1: reward=-20, steps=1022, speed=0.0 f/s, elapsed=0:01:03
Episode 2: reward=-20, steps=951, speed=0.0 f/s, elapsed=0:01:03
Episode 3: reward=-19, steps=945, speed=0.0 f/s, elapsed=0:01:03
Episode 4: reward=-18, steps=1026, speed=0.0 f/s, elapsed=0:01:03
Episode 5: reward=-21, steps=836, speed=0.0 f/s, elapsed=0:01:03
Episode 6: reward=-20, steps=1025, speed=0.0 f/s, elapsed=0:01:03
Episode 7: reward=-21, steps=983, speed=0.0 f/s, elapsed=0:01:03
Episode 8: reward=-20, steps=913, speed=0.0 f/s, elapsed=0:01:03
Episode 9: reward=-21, steps=837, speed=0.0 f/s, elapsed=0:01:03
Episode 10: reward=-21, steps=942, speed=0.0 f/s, elapsed=0:01:03
Episode 11: reward=-20, steps=988, speed=71.4 f/s, elapsed=0:01:09
Episode 12: reward=-20, steps=950, speed=71.4 f/s, elapsed=0:01:23
Episode 13: reward=-20, steps=960, speed=71.4 f/s, elapsed=0:01:36
Episode 14: reward=-20, steps=837, speed=71.4 f/s, elapsed=0:01:48
Episode 15: reward=-20, steps=862, speed=71.3 f/s, elapsed=0:02:00
Episode 16:

Engine run is terminating due to exception: .


KeyboardInterrupt: 